In [1]:
import json
import math
import os
from pprint import pprint
import numpy as np
import tensorflow as tf


# Loading data from memory
## Create the dataset

In [4]:
N_POINTS = 10
X = tf.constant(range(N_POINTS), dtype=tf.float32)
Y = 2 * X + 10
print(X)
print(Y)

tf.Tensor([0. 1. 2. 3. 4. 5. 6. 7. 8. 9.], shape=(10,), dtype=float32)
tf.Tensor([10. 12. 14. 16. 18. 20. 22. 24. 26. 28.], shape=(10,), dtype=float32)


Implement a function take input and return a tf.data.Dataset:
* Number of passes over the dataset to train on: epochs
* size of the batches: batch_size

** The last batch may not contain the exact number of elements as batch_size
** To discard the last batch:
```python
dataset = dataset.batch(batch_size, drop_remainder=True)
```

In [5]:
def create_dataset(X, Y, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X,Y))
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    return dataset

In [7]:
BATCH_SIZE = 3
EPOCH = 2

dataset = create_dataset(X,Y,EPOCH,BATCH_SIZE)

for i, (x,y) in enumerate(dataset):
    print("x:{}".format(x.numpy()), "y:{}".format(y.numpy()))
    assert len(x) == BATCH_SIZE
    assert len(y) == BATCH_SIZE

x:[0. 1. 2.] y:[10. 12. 14.]
x:[3. 4. 5.] y:[16. 18. 20.]
x:[6. 7. 8.] y:[22. 24. 26.]
x:[9. 0. 1.] y:[28. 10. 12.]
x:[2. 3. 4.] y:[14. 16. 18.]
x:[5. 6. 7.] y:[20. 22. 24.]
